In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import regex as re
from jellyfish import jaro_similarity
from tqdm import tqdm

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep

In [3]:
filename = "covid_philippines"
path = "datasets/" + filename + "/" + filename + ".csv"
video_df = pd.read_csv(path).drop("Unnamed: 0", axis=1)

In [4]:
video_df.head()

,video_id,video_title,description,video_dop,view_count,like_count,comment_count,channel_id,video_transcript
0,aLZ85hb4wjE,Normally crowded streets of Philippine capital...,Subscribe to our YouTube channel for free here...,2020-04-02,325303,3291,619,UC4SUWizzKc1tptprBkWjX2Q,NaN
1,sYI97jv-pZg,"PH records 2,725 new COVID cases from Dec. 12 ...",The World Tonight: The daily average of the Ph...,2023-12-18,2325,15,3,UCvi6hEzLM-Z_unKPSuuzKvg,NaN
2,DWxIvQlpJK8,Metro Manila to be placed on lockdown due to c...,Subscribe: https://www.youtube.com/@Rappler/\n...,2020-03-12,107238,750,19,UCdnZdQxYXnbN4uWJg96oGxw,NaN
3,3YFpjgIQqEo,WATCH: DOH Usec. Tayag on rise of COVID-19 cas...,Dateline Philippines: Karmina Constantino talk...,2023-12-07,9808,81,17,UCvi6hEzLM-Z_unKPSuuzKvg,NaN
4,cPVE7QGS7As,Philippines faces worst COVID-19 crisis in Sou...,The Philippines has the highest number of coro...,2020-08-19,127851,906,533,UCNye-wNBqNL5ZzHSJj3l8Bg,joey dejos is checking whether his cash regis...


In [5]:
video_id_list = video_df["video_id"].tolist()

In [6]:
for video_id in video_id_list[0:5]:
    print(f"https://www.youtube.com/watch?v={video_id}")

https://www.youtube.com/watch?v=aLZ85hb4wjE
https://www.youtube.com/watch?v=sYI97jv-pZg
https://www.youtube.com/watch?v=DWxIvQlpJK8
https://www.youtube.com/watch?v=3YFpjgIQqEo
https://www.youtube.com/watch?v=cPVE7QGS7As


---

In [11]:
def get_backlinks(video_id_list):
    backlinks = {}
    
    driver = webdriver.Firefox()
    driver.get("https://www.thehoth.com/backlinks-checker/")
    driver.maximize_window()
    
    captcha_passed = False
    
    
    try:
        for video_id in video_id_list:
            # print("Inputting video link")
            # Wait for page to load and locate textbox
            # print("Waiting for frame")
            WebDriverWait(driver, 300).until(
                    EC.frame_to_be_available_and_switch_to_it((By.ID, "hothtools"))
                )

            # Wait for textbox to be clickable and then click
            # print("Waiting for textbox")
            WebDriverWait(driver, 300).until(
                    EC.element_to_be_clickable((By.ID, "targeturl"))
                )

            # Input video link
            targeturl = driver.find_element(By.ID, "targeturl")
            targeturl.click()
            targeturl.send_keys(Keys.CONTROL + "A")
            targeturl.send_keys(Keys.BACKSPACE)
            targeturl.send_keys(f"https://www.youtube.com/watch?v={video_id}")

            # Select 'This Exact URL'
            dropdown = driver.find_element(By.ID, "mode")
            dropdown.click()
            option = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/select/option[3]")
            option.click()

            # Submit
            submit = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/form/div[2]/button")
            submit.click()

            sleep(5)

            try:
                if not captcha_passed:
                    # print("Solving CAPTCHA")
                    # In case of Captcha page
                    iframes = driver.find_elements(By.TAG_NAME, "iframe")
                    cont = False
                    for iframe in iframes:
                        try:
                            driver.switch_to.frame(iframe)
                            checkbox = driver.find_element(By.ID, "recaptcha-anchor")
                            checkbox.click()
                            cont = True
                            break
                        except:
                            # print("Not Found")
                            driver.switch_to.default_content()

                    if not cont:
                        raise Exception("No CAPTCHA iframe found.")

                    # Giving enough time to manually solve CAPTCHA puzzle
                    sleep(15)

                    driver.switch_to.default_content()
                    driver.switch_to.frame("hothtools")
                    submit = driver.find_element(
                        By.XPATH, "//*[@id='submit']"
                    )
                    submit.click()
                    captcha_passed = True
                    sleep(3)
            except:
                pass
            finally:
                # print("Getting results")
                # Get external backlinks
                driver.switch_to.default_content()
                driver.switch_to.frame("hothtools")
                try:
                    WebDriverWait(driver, 300).until(
                        EC.element_to_be_clickable((By.ID, "targeturl"))
                    )
                    
                    result = driver.find_element(
                        By.XPATH, "/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[1]/div[2]"
                    ).text
                except:
                    result = 0
                finally:
                    backlinks[video_id] = result
                    driver.switch_to.default_content()
    '''
    except:
        print("Error raise by Selenium Webdriver. Ending process and saving data.")
    # '''
    
    '''
    finally:
        driver.close()
    # '''
    
    return backlinks

In [12]:
backlinks = get_backlinks(video_id_list)

Error raise by Selenium Webdriver. Ending process and saving data.


In [13]:
print(f"Completed: {len(backlinks)}/{len(video_id_list)}")

Completed: 82/150


In [14]:
backlinks

{'aLZ85hb4wjE': '4',
 'sYI97jv-pZg': '1',
 'DWxIvQlpJK8': 0,
 '3YFpjgIQqEo': 0,
 'cPVE7QGS7As': '24',
 'pMUumjHY3tw': '1',
 'dIsaz_XlmTw': '1',
 'CEIrzjA8euQ': '2',
 'iOE6rAY8l-k': '10',
 'Mxf8uGFcqSE': '1',
 'E5F3xA_zkFc': '23',
 'MQ5aYS4YFlQ': '11',
 'qERZgLv3MKg': '5',
 'E56W-5xVOss': '3',
 '2fRQ8OsqOLs': '1',
 'c1oU8U05puY': '2',
 '_a-rQYfsCck': 0,
 '8dTelszbObM': '47',
 'lw16DeB6zns': '29',
 'RhDHGgo4yZg': '2',
 'jEINZXA_ujo': '1',
 'wJKPv-XmArQ': '27',
 '587N9bJ5J5k': '28',
 'nTUWK8vufOk': '18',
 '1psSvU1km0I': '1',
 'oChS6LbuluU': 0,
 'liEUC1_l8e8': '17',
 'PP3Yu-ro1tA': 0,
 'LzymJ2xZhho': 0,
 'SEHcakm-fAc': '3',
 'NQeI1CRCqeo': 0,
 'CM_3Vn3ksr8': 0,
 'sdsz-t540WI': '35',
 'oy0wHScCPds': 0,
 '7SKGXkZKjV8': 0,
 'o1kPskxFkQ8': 0,
 'Fd3HcncV6Zk': '21',
 '-n9Ks3VTub4': '2',
 'MmyIvf7bEGc': 0,
 'xV1oR-RbOGU': '18',
 'aoNQUUQno00': '2',
 '3ZXR2eARmuQ': '1',
 'a3-AiHSyTIU': 0,
 'cshl1Gzj2Z0': 0,
 'yBNSwAVSei8': '3',
 '-qFcO_onBdA': '1',
 'kn6DKdInYXk': '2',
 'ZLT6L3PHz78': 0,
 'sH_YoV-